# **Homework 3: Function-based RL**
#### **Created by 65340500058 Anuwit Intet**

## **Learning Objectives:**

- Understand how function approximation works and how to implement it.

- Understand how policy-based RL works and how to implement it.

- Understand how advanced RL algorithms balance exploration and exploitation.

- Be able to differentiate RL algorithms based on stochastic or deterministic policies, as well as value-based, policy-based, or Actor-Critic approaches.

- Gain insight into different reinforcement learning algorithms, including Linear Q-Learning, Deep Q-Network (DQN), the REINFORCE algorithm, and the Actor-Critic algorithm. Analyze their strengths and weaknesses.


In [ ]:
import torch

## <font color="pink">**Part 1: Understanding the Algorithm**</font>

In this homework, you have to implement 4 different function approximation-based RL algorithms:

- Linear Q-Learning
 
- Deep Q-Network (DQN)

- REINFORCE algorithm

- One algorithm chosen from the following Actor-Critic methods:

    - Deep Deterministic Policy Gradient (DDPG)

    - Advantage Actor-Critic (A2C)

    - Proximal Policy Optimization (PPO)
    
    - Soft Actor-Critic (SAC)

For each algorithm, describe whether it follows a value-based, policy-based, or Actor-Critic approach, specify the type of policy it learns (stochastic or deterministic), identify the type of observation space and action space (discrete or continuous), and explain how each advanced RL method balances exploration and exploitation.

- it follows a value-based, policy-based, or Actor-Critic approach

- the type of policy it learns (stochastic or deterministic)

- the type of observation space and action space (discrete or continuous)

- how each advanced RL method balances exploration and exploitation

### <font color="yellow">**Linear Q-Learning**</font>

- About Linear Q-Learning
  - Linear Q-Learning is a value-based approach. It sometimes learns a function Q(s, a) that is used to determine the method by selecting the maximum Q-value action.

  - This algorithm uses the deterministic policy because Linear Q-Learning uses a ε-greedy policy which argmax Q-value, not uses the probability.

  - Linear Q-Learning is applied to continuous observation space (because it uses input feature vectors). But the action space must be discrete because it must compute $max⁡_Q(s,a)$, which must look at all actions. 

  - To balance Exploration vs Exploitation, Linear Q-Learning uses a ε-greedy policy, i.e. random action with probability ε and greedy action with probability 1−ε.

- In Linear Q-Learning, Q-Function is estimate by

  $$
  Q(s,a) = \phi(s,a)^T w
  $$

  where:

  - $\phi(s,a)$ is feature vector of state-action pair  
  - $w$ is weight vector

- And update weight by this,

  $$
  w \leftarrow w + \alpha \cdot \delta \cdot \phi(s, a)
  $$

  where: 
  - $\delta = r + \gamma \max_{a'} Q(s', a') - Q(s, a)$ is TD error
  - $\alpha$ is learning rate


**🎮 ตัวอย่าง Linear Q-Learning บน CartPole**

- **State**: เวกเตอร์ขนาด 4:  
  $$
  s = [x, \dot{x}, \theta, \dot{\theta}]
  $$
- **Action space**: มี 2 ค่า (discrete):
  - `0` = push cart ไปซ้าย
  - `1` = push cart ไปขวา

---

เราจะประมาณ $Q(s, a)$ ด้วยฟังก์ชันเชิงเส้นแบบนี้:

$$
Q(s, a) = w_a^T s
$$

โดยที่:
- $w_0$, $w_1$ คือ weight vectors สำหรับ action 0 และ 1 ตามลำดับ
- หรือรวมเป็น matrix $W \in \mathbb{R}^{2 \times 4}$

---

- $s = [0.0, 0.5, 0.05, -0.2]$
- $W = \begin{bmatrix} 0.1 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.1 & 0.0 & 0.0 \end{bmatrix}$
- เลือก action $a = 1$
- ได้ reward $r = 1$
- next state: $s' = [0.01, 0.55, 0.045, -0.18]$
- $\alpha = 0.1$, $\gamma = 0.99$

---

**1. คำนวณ $Q(s, a)$**

$$
Q(s, a=1) = w_1^T s = 0.0*0.0 + 0.1*0.5 + 0.0*0.05 + 0.0*(-0.2) = 0.05
$$

---

**2. คำนวณ $\max_{a'} Q(s', a')$**

$$
Q(s', 0) = w_0^T s' = 0.1*0.01 = 0.001 \\
Q(s', 1) = w_1^T s' = 0.1*0.55 = 0.055 \\
\Rightarrow \max_{a'} Q(s', a') = 0.055
$$

---

**3. คำนวณ TD Error**

$$
\delta = r + \gamma \cdot \max_{a'} Q(s', a') - Q(s, a) \\
= 1 + 0.99 \cdot 0.055 - 0.05 = 1.00445
$$

---

**4. อัปเดต weight**

เฉพาะ $w_1$:

$$
w_1 \leftarrow w_1 + \alpha \cdot \delta \cdot s \\
= [0.0, 0.1, 0.0, 0.0] + 0.1 \cdot 1.00445 \cdot [0.0, 0.5, 0.05, -0.2] \\
= [0.0, 0.1502, 0.005, -0.0201]
$$

---

### <font color="yellow">**Deep Q-Network (DQN)**</font>

🔸 DQN คืออะไร?

DQN แก้ปัญหานี้โดยใช้ deep neural network มาแทนตาราง Q โดยประมาณฟังก์ชัน: $Q(s,a;θ)$


โดยที่:

- θ คือพารามิเตอร์ของ neural network

- input = สถานะ s (เช่น ภาพจากเกม Atari)

- output = ค่า Q สำหรับทุก action

🔧 การฝึก DQN ประกอบด้วยเทคนิคหลัก 3 อย่าง:

- Experience Replay
    - เก็บประสบการณ์ $(s, a, r, s', done)$ ลงใน buffer
    - แล้วสุ่ม mini-batch มาใช้ฝึก เพื่อ decorrelate ข้อมูล

- Target Network
    - ใช้ network แยกอีกอันชื่อ target network QtargetQtarget​ เพื่อคำนวณค่าที่ใช้เป็น target:y=r+γmax⁡a′Qtarget(s′,a′)
    - แล้ว update เฉพาะ network หลัก Q เป็นระยะ

- Fixed Action Space
    - Action space ต้อง discrete เพื่อให้คำนวณ max⁡aQ(s,a)maxa​Q(s,a) ได้ง่าย

🔁 สรุป Training Loop:

- รับสถานะ s

- เลือก action จาก Q-network ด้วย $\epsilon$-greedy

- ทำ action → ได้ reward r, next state s′

- เก็บ transition ลง replay buffer

- สุ่ม batch จาก buffer → คำนวณ TD error

- อัปเดต θ โดย gradient descent

✅ คำถามย่อย
1. ❓ It follows a value-based, policy-based, or Actor-Critic approach?

คำตอบ:
🔹 DQN เป็น value-based approach เพราะมันเรียนรู้ฟังก์ชันค่า Q(s,a)Q(s,a) โดยตรง แล้ว derive policy จากการเลือก action ที่ให้ค่า Q สูงสุด

2. ❓ The type of policy it learns (stochastic or deterministic)?

คำตอบ:
🔸 นโยบายที่เรียนรู้คือ deterministic
เพราะมันเลือก action ด้วย:
a∗=arg⁡max⁡aQ(s,a)

แต่ในระหว่าง training จะใช้: $\epsilon$ -greedy policy → stochastic เพื่อการ explore

ดังนั้น:

Phase	Type of policy
Training	Stochastic (for exploration)
Deployment	Deterministic (greedy action)

3. ❓ The type of observation space and action space?

คำตอบ:
- Observation Space	Continuous ได้ เช่น รูปภาพ, เวกเตอร์ ฯลฯ (ผ่าน neural net)
- Action Space	Discrete เท่านั้น (เพราะ DQN ต้องคำนวณ $\max_a Q(s, a)$)

หากต้องการใช้กับ continuous action ต้องเปลี่ยนไปใช้ DDPG, SAC, หรือ TD3 แทน

4. ❓ How does this method balance exploration and exploitation?

คำตอบ:
- DQN ใช้ ε-greedy strategy ในการ balance:

    - โอกาสสุ่ม (explore) = $\epsilon$

    - โอกาสเลือก action ที่ดีที่สุด (exploit) = $1 - \epsilon$

- โดยปรับ $\epsilon$ ตาม schedule เช่น:

    - เริ่มต้นที่ $\epsilon = 1.0$

    - ลดลงทีละน้อยถึงเช่น $\epsilon = 0.1$ หรือ $\epsilon = 0.01$

### <font color="yellow">**REINFORCE algorithm**</font>

### <font color="yellow">**Deep Deterministic Policy Gradient (DDPG)**</font>

### <font color="yellow">**Advantage Actor-Critic (A2C)**</font>

### <font color="yellow">**Proximal Policy Optimization (PPO)**</font>

# 🧠 Proximal Policy Optimization (PPO)

## ✅ แนวคิดหลักของ PPO

PPO คือหนึ่งใน Policy Gradient algorithms ที่ได้รับความนิยมสูงมาก ซึ่งพัฒนาโดย OpenAI โดยมีเป้าหมายหลักเพื่อ:

- ทำให้การอัปเดตนโยบาย (Policy) มีเสถียรภาพ (stable)
- เรียนรู้จากข้อมูลที่มาจาก policy ปัจจุบัน (on-policy)
- หลีกเลี่ยงการอัปเดตนโยบายแบบ “แรงเกินไป” ซึ่งอาจทำให้ performance แย่ลง

---

## 🏗 โครงสร้าง PPO

PPO ใช้แนวทาง **Actor-Critic** ซึ่งประกอบด้วย:

- **Actor**: เรียนรู้นโยบาย $\pi_\theta(a|s)$ → ใช้เลือก action
- **Critic**: ประเมิน value ของ state หรือ action เช่น $V(s)$ หรือ $Q(s,a)$ → ใช้คำนวณ advantage

---

## 🔁 หลักการทำงานของ PPO (Step-by-Step)

### 1. **Collect Trajectories**
- ให้ agent วิ่งใน environment ตาม policy ปัจจุบัน
- เก็บข้อมูล: $(s_t, a_t, r_t, \log \pi(a_t|s_t), done)$
- รอจนได้ rollout ครบ (เช่น 2048 steps หรือ 1 episode)

---

### 2. **Compute Returns & Advantages**
- คำนวณ **Monte Carlo return** หรือใช้ **GAE (Generalized Advantage Estimation)**:
  
  ```math
  A_t = \delta_t + (\gamma \lambda) \delta_{t+1} + ... ≈ R_t - V(s_t)

### 3. Surrogate Objective with Clipping

- PPO ใช้ surrogate loss function เพื่อควบคุมการอัปเดตของนโยบาย:
- rt(θ)=πθ(at∣st)πθold(at∣st)
- LCLIP(θ)=Et[min⁡(rt(θ)At,clip(rt(θ),1−ϵ,1+ϵ)At)]
- ถ้า $r_t$ เบี่ยงเบนจาก 1 มากเกินไป → จะถูก clip ไว้ เพื่อป้องกัน policy เปลี่ยนแปลงเร็วเกินไป

### 4. Update Policy and Value Function

- อัปเดต Actor ด้วย loss จาก surrogate objective

- อัปเดต Critic ด้วย MSE loss ระหว่าง $V(s_t)$ กับ return

### 5. Repeat Training for Multiple Epochs

- ใช้ข้อมูล rollout เดิมฝึกได้หลายรอบ (เช่น 4-10 epochs)

- ทำให้ sample efficient โดยไม่ต้องใช้ replay buffer

### <font color="yellow">**Soft Actor-Critic (SAC)**</font>

## <font color="pink">**Part 2: Setting up Cart-Pole Agent**</font>

Similar to the previous homework, you will implement a common components that will be the same in most of the function approximation-based RL in the RL_base_function.py.The core components should include, but are not limited to:

### <font color="orange">**1. RL Base class**</font>

- This class should include:

    - Constructor (__init__) to initialize the following parameters:

        - Number of actions: The total number of discrete actions available to the agent.

        - Action range: The minimum and maximum values defining the range of possible actions.

        - Discretize state weight: Weighting factor applied when discretizing the state space for learning.

        - Learning rate: Determines how quickly the model updates based on new information.

        - Initial epsilon: The starting probability of taking a random action in an ε-greedy policy.

        - Epsilon decay rate: The rate at which epsilon decreases over time to favor exploitation over exploration.

        - Final epsilon: The lowest value epsilon can reach, ensuring some level of exploration remains.

        - Discount factor: A coefficient (γ) that determines the importance of future rewards in decision-making.

        - Buffer size: Maximum number of experiences the buffer can hold.

        - Batch size: Number of experiences to sample per batch.

    - Core Functions

        - scale_action(): scale the action (if it is computed from the sigmoid or softmax function) to the proper length.

        - decay_epsilon(): Decreases epsilon over time and returns the updated value.

- Additional details about these functions are provided in the class file. You may also implement additional functions for further analysis.

#### <font color="yellow">**scale_action()**</font>

In [ ]:
def scale_action(self, action):
    """
    Maps a discrete action in range [0, n] to a continuous value in [action_min, action_max].

    Args:
        action (int): Discrete action in range [0, n].
        n (int): Number of discrete actions (inclusive range from 0 to n).
    
    Returns:
        torch.Tensor: Scaled action tensor.
    """
    # ========= put your code here ========= #

    # Unpack the minimum and maximum values of the action range
    action_min, action_max = self.action_range

    # Scale the discrete action index (0 to num_of_action-1) to a continuous value within [action_min, action_max]
    scaled = action_min + (action / (self.num_of_action - 1)) * (action_max - action_min)

    # Check if the scaled value is already a torch.Tensor
    if isinstance(scaled, torch.Tensor):
        # If yes, detach it from any computation graph and convert to float32
        return scaled.clone().detach().to(dtype=torch.float32)
    else:
        # Otherwise, convert it into a torch.Tensor of type float32
        return torch.tensor(scaled, dtype=torch.float32)

    # ====================================== #

#### <font color="yellow">**decay_epsilon()**</font>

In [ ]:
def decay_epsilon(self):
    """
    Decay epsilon value to reduce exploration over time.
    """
    # ========= put your code here ========= #
    # Decay the exploration rate (epsilon) by multiplying with epsilon_decay,
    # but ensure it doesn't go below the minimum value (final_epsilon)
    self.epsilon = max(self.final_epsilon, self.epsilon * self.epsilon_decay)
    # ====================================== #

### <font color="orange">**2. Replay Buffer Class**</font>



- A class use to store state, action, reward, next state, and termination status from each timestep in episode to use as a dataset to train neural networks. This class should include:

    - Constructor (__init__) to initialize the following parameters:

        - memory: FIFO buffer to store the trajectory within a certain time window.

        - batch_size: Number of data samples drawn from memory to train the neural network.

    - Core Functions

        - add(): Add state, action, reward, next state, and termination status to the FIFO buffer. Discard the oldest data in the buffer

        - sample(): Sample data from memory to use in the neural network training.

    - <font color="orange">**Note that some algorithms may not use all of the data mentioned above to train the neural network.**</font>


#### <font color="yellow">**add()**</font>

#### <font color="yellow">**sample()**</font>

### <font color="orange">**3. Algorithm folder**</font>

- This folder should include:

    - Linear Q Learning class

    - Deep Q-Network class

    - REINFORCE Class

    - One class chosen from the Part 1.

- Each class should inherit from the RL Base class in RL_base_function.py and include:

    - A constructor which initializes the same variables as the class it inherits from.

    - Superclass Initialization (super().__init__()).

    - An update() function that updates the agent’s learnable parameters and advances the training step.

    - A select_action() function select the action according to current policy.

    - A learn() function that train the regression or neural network.

#### <font color="yellow">**Linear Q-Learning class**</font>

#### <font color="yellow">**Deep Q-Network (DQN) class**</font>

#### <font color="yellow">**REINFORCE class**</font>

#### <font color="yellow">**DDPG class**</font>

#### <font color="yellow">**A2C class**</font>

#### <font color="yellow">**PPO class**</font>

#### <font color="yellow">**SAC class**</font>

## <font color="pink">**Part 3: Trainning & Playing to stabilize Cart-Pole Agent**</font>

You need to implement the training loop in train script and main() in the play script (in the "Can be modified" area of both files). Additionally, you must collect data, analyze results, and save models for evaluating agent performance.

- Training the Agent

    - Stabilizing Cart-Pole Task

        ```python
        python scripts/Function_based/train.py --task Stabilize-Isaac-Cartpole-v0
        ```

    - Swing-up Cart-Pole Task (Optional)

        ```python
        python scripts/Function_based/train.py --task SwingUp-Isaac-Cartpole-v0
        ```

- Playing

    - Stabilize Cart-Pole Task

        ```python
        python scripts/Function_based/play.py --task Stabilize-Isaac-Cartpole-v0
        ``` 

    - Swing-up Cart-Pole Task (Optional)

        ```python
        python scripts/Function_based/play.py --task SwingUp-Isaac-Cartpole-v0 
        ```

### <font color="yellow">**train.py**</font>

### <font color="yellow">**play.py**</font>

## <font color="pink">**Part 4: Evaluate Cart-Pole Agent performance**</font>

You must evaluate the agent's performance in terms of learning efficiency (i.e., how well the agent learns to receive higher rewards) and deployment performance (i.e., how well the agent performs in the Cart-Pole problem). Analyze and visualize the results to determine:

- Which algorithm performs best?

- Why does it perform better than the others?


### <font color="yellow">**In term of learning efficiency**</font>

### <font color="yellow">**In term of deployment performance**</font>